In [38]:
import pandas as pd
import sqlite3
import matplotlib as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
from scipy.stats.stats import pearsonr

In [2]:
df_gross = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")

In [5]:
df_gross['foreign_gross'] = df_gross['foreign_gross'].str.replace(',', '')

In [6]:
df_gross['foreign_gross'] = df_gross['foreign_gross'].astype(float)

In [7]:
df_gross.dropna(subset=['domestic_gross', 'foreign_gross'], inplace=True)

In [8]:
df_gross['total_gross']= df_gross['domestic_gross'] + df_gross['foreign_gross']
df_gross

,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08
...,...,...,...,...,...,...
3275,I Still See You,LGF,1400.0,1500000.0,2018,1.501400e+06
3286,The Catcher Was a Spy,IFC,725000.0,229000.0,2018,9.540000e+05
3309,Time Freak,Grindstone,10000.0,256000.0,2018,2.660000e+05
3342,Reign of Judges: Title of Liberty - Concept Short,Darin Southa,93200.0,5200.0,2018,9.840000e+04


In [9]:
studio_group = df_gross.groupby('studio')
avg_total = studio_group['total_gross'].mean()
sorted_avg_total = avg_total.sort_values(ascending=False)
top10studios = pd.DataFrame(sorted_avg_total).head(10)
top10studios

,total_gross
studio,
HC,8.703000e+08
P/DW,5.076500e+08
BV,4.249075e+08
GrtIndia,2.542000e+08
WB,2.368577e+08
WB (NL),2.344114e+08
Fox,2.311968e+08
Sony,2.133782e+08
Par.,2.068532e+08


In [10]:
df_movinfo = pd.read_csv("./zippedData/rt.movie_info.tsv.gz", delimiter="\t")

In [11]:
df_movinfo.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [12]:
df_movinfo['dvd_date'] = pd.to_datetime(df_movinfo['dvd_date'])
df_movinfo['theater_date'] = pd.to_datetime(df_movinfo['theater_date'])

In [13]:
df_reviews = pd.read_csv("./zippedData/rt.reviews.tsv.gz", delimiter="\t", encoding='latin1')

In [14]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [15]:
df_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [16]:
df_movies = pd.read_csv("./zippedData/tmdb.movies.csv.gz")

In [17]:
df_movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [18]:
df_budgets = pd.read_csv("./zippedData/tn.movie_budgets.csv.gz")

In [19]:
df_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [20]:
df_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [21]:
df_budgets['release_date'] = pd.to_datetime(df_budgets['release_date'])

In [22]:
df_budgets['production_budget'] = df_budgets['production_budget'].replace('[\$\,]', '', regex=True).astype(float)
df_budgets['domestic_gross'] = df_budgets['domestic_gross'].replace('[\$\,]', '', regex=True).astype(float)
df_budgets['worldwide_gross'] = df_budgets['worldwide_gross'].replace('[\$\,]', '', regex=True).astype(float)
df_budgets['profit'] = df_budgets['worldwide_gross'] - df_budgets["production_budget"]
df_budgets = df_budgets.sort_values(by="profit", ascending=False)
#df_budgets = df_budgets.head(10)
#df_budgets = df_budgets.loc[df_budgets['release_date'] >= "2011"]
df_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit
0,1,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09
42,43,1997-12-19,Titanic,200000000.0,659363944.0,2.208208e+09,2.008208e+09
6,7,2018-04-27,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,1.748134e+09
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,1.747311e+09
33,34,2015-06-12,Jurassic World,215000000.0,652270625.0,1.648855e+09,1.433855e+09
...,...,...,...,...,...,...,...
352,53,2001-04-27,Town & Country,105000000.0,6712451.0,1.036477e+07,-9.463523e+07
341,42,2019-06-14,Men in Black: International,110000000.0,3100000.0,3.100000e+06,-1.069000e+08
193,94,2011-03-11,Mars Needs Moms,150000000.0,21392758.0,3.954976e+07,-1.104502e+08
194,95,2020-12-31,Moonfall,150000000.0,0.0,0.000000e+00,-1.500000e+08


In [43]:
stats.ttest_ind(df_budgets['production_budget'], df_budgets['profit'])

Ttest_indResult(statistic=-14.167552336603405, pvalue=3.4717591209672177e-45)

In [46]:
print(df_budgets['production_budget'].corr(df_budgets['profit']))

0.6087521471718846


In [48]:
conn = sqlite3.connect('./zippedData/im.db')
qr = pd.read_sql("""
SELECT *
FROM sqlite_master


""", conn)
qr

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [25]:
q = pd.read_sql("""
SELECT *, averagerating, numvotes
FROM movie_basics
JOIN(movie_ratings)
USING(movie_id)
ORDER BY averagerating DESC, numvotes DESC

""", conn)
q

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,averagerating,numvotes
0,tt9715646,Renegade,Renegade,2019,NaN,Documentary,10.0,20,10.0,20
1,tt4960818,Revolution Food,Revolution Food,2015,70.0,Documentary,10.0,8,10.0,8
2,tt5089804,Fly High: Story of the Disc Dog,Fly High: Story of the Disc Dog,2019,65.0,Documentary,10.0,7,10.0,7
3,tt4109192,I Was Born Yesterday!,I Was Born Yesterday!,2015,31.0,Documentary,10.0,6,10.0,6
4,tt5344358,All Around Us,All Around Us,2019,NaN,Documentary,10.0,6,10.0,6
...,...,...,...,...,...,...,...,...,...,...
73851,tt5452328,Tachiiri kinshi Haittara shinu? Norowareta 5 hen,Tachiiri kinshi Haittara shinu? Norowareta 5 hen,2016,61.0,Horror,1.0,6,1.0,6
73852,tt2447822,Momok jangan cari pasal!,Momok jangan cari pasal!,2012,85.0,Comedy,1.0,5,1.0,5
73853,tt6792126,Jak se mori revizori,Jak se mori revizori,2018,NaN,Comedy,1.0,5,1.0,5
73854,tt8426154,Pup Scouts,Pup Scouts,2018,72.0,Animation,1.0,5,1.0,5


In [26]:
df_sql = q

In [27]:
df_sql.dropna(subset=['runtime_minutes'], inplace=True)
df_sql.dropna(subset=['genres'], inplace=True)

In [28]:
df_sql.isna().sum()

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
averagerating      0
numvotes           0
dtype: int64

In [29]:
df_sql = df_sql.loc[:, ~df_sql.columns.duplicated()]
df_sql

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
1,tt4960818,Revolution Food,Revolution Food,2015,70.0,Documentary,10.0,8
2,tt5089804,Fly High: Story of the Disc Dog,Fly High: Story of the Disc Dog,2019,65.0,Documentary,10.0,7
3,tt4109192,I Was Born Yesterday!,I Was Born Yesterday!,2015,31.0,Documentary,10.0,6
5,tt7227500,Ellis Island: The Making of a Master Race in A...,Ellis Island: The Making of a Master Race in A...,2018,70.0,"Documentary,History",10.0,6
6,tt7259300,Calamity Kevin,Calamity Kevin,2019,77.0,"Adventure,Comedy",10.0,6
...,...,...,...,...,...,...,...,...
73850,tt3717376,A Time to Speak,A Time to Speak,2014,60.0,Documentary,1.0,6
73851,tt5452328,Tachiiri kinshi Haittara shinu? Norowareta 5 hen,Tachiiri kinshi Haittara shinu? Norowareta 5 hen,2016,61.0,Horror,1.0,6
73852,tt2447822,Momok jangan cari pasal!,Momok jangan cari pasal!,2012,85.0,Comedy,1.0,5
73854,tt8426154,Pup Scouts,Pup Scouts,2018,72.0,Animation,1.0,5


In [30]:
df_sql.value_counts('genres')

genres
Drama                       10189
Documentary                  9149
Comedy                       4604
Comedy,Drama                 2451
Horror                       2232
                            ...  
Biography,Drama,News            1
Biography,Drama,Western         1
Biography,Family,Fantasy        1
Biography,Family,History        1
Comedy,Documentary,War          1
Length: 906, dtype: int64

In [31]:
model = ols('averagerating ~ C(genres)', data=df_sql).fit()

# Perform ANOVA and print the table
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

                  sum_sq       df          F  PR(>F)
C(genres)   38667.837093    905.0  27.363675     0.0
Residual   101203.539400  64814.0        NaN     NaN


In [32]:
genre_group = df_sql.groupby('genres')
genre_avg_rating = genre_group['averagerating'].mean()
genre_avg_votes = genre_group['numvotes'].mean()
genre_avg_rating


genres
Action                        5.665187
Action,Adult,Comedy           5.900000
Action,Adventure              5.059677
Action,Adventure,Animation    6.548734
Action,Adventure,Biography    7.105556
                                ...   
Thriller                      5.603727
Thriller,War                  5.650000
Thriller,Western              6.300000
War                           5.746667
Western                       4.939062
Name: averagerating, Length: 906, dtype: float64

In [34]:
df_genres = df_sql['genres'].str.get_dummies(sep=',')
df_genres_weighted = df_genres.mul(df_sql['numvotes'], axis=0)  # Multiply the genres DataFrame by the number of votes
df_genres_weighted = df_genres_weighted.mul(df_sql['averagerating'], axis=0)  # Multiply the result by the ratings
df_genres_weighted = df_genres_weighted.replace(0, np.nan)  # Replace 0s with NaNs to not affect the mean
genre_means_weighted = df_genres_weighted.sum() / df_genres.mul(df_sql['numvotes'], axis=0).sum()

In [35]:
df_genre_means = genre_means_weighted.to_frame(name='genre_average_rating')
df_genre_means.reset_index(inplace=True)
df_genre_means.rename(columns={'index': 'genres'}, inplace=True)
df_genre_means = df_genre_means.sort_values(by='genre_average_rating', ascending=False)

In [36]:
df_genre_means

,genres,genre_average_rating
21,Short,8.800000
17,News,7.640460
25,Western,7.519219
4,Biography,7.475118
12,History,7.358505
7,Documentary,7.298141
22,Sport,7.291363
3,Animation,7.265119
14,Music,7.131506
20,Sci-Fi,7.120721


In [37]:
stats.ttest_ind(df_genre_means['genre_average_rating'], df_sql['runtime_minutes'])

Ttest_indResult(statistic=-2.1410512608121604, pvalue=0.0322735704355956)

In [42]:
print(df_genre_means['genre_average_rating'].corr(df_sql['runtime_minutes']))

0.05130152014552732
